In [ ]:
MODEL_NAME = 'feedbacken'
PYTHONPATH = 'PYTHONPATH=.:/kaggle:/kaggle/input/pikachu/utils:/kaggle/input/pikachu/third:$PYTHONPATH'

In [ ]:
import sys
import os  
os.environ["TOKENIZERS_PARALLELISM"] = "false"

!rm -rf ./src
!cp -rf ../input/{MODEL_NAME} ./src
if os.path.exists('/kaggle'):
  sys.path.append('/kaggle/input/pikachu/utils')
  sys.path.append('/kaggle/input/pikachu/third')
  sys.path.append('.')
!ls ../input

In [ ]:
!pip install -q icecream --no-index --find-links=file:///kaggle/input/icecream/ 
!pip install -q datasets --no-index --find-links=file:///kaggle/input/hug-datasets/ 
!pip install -q pymp-pypi --no-index --find-links=file:///kaggle/input/pymp-pypi/pymp-pypi-0.4.5/dist
!wandb disabled

In [ ]:
from gezi.common import *
gezi.beijing_time_str()

In [ ]:
FOLDS = 2
NUM_MODELS = 23
INDEXES_LIST = [
    list(range(NUM_MODELS * FOLDS)),
]
WEIGHTS = [1] * NUM_MODELS * FOLDS

In [ ]:
TEST_MODE = 0
# TEST_MODE = 1
TEST_MODE

In [ ]:
%%writefile ./src/infer.py 
#!/usr/bin/env python 
# -*- coding: utf-8 -*-
# ==============================================================================
#          \file   infer.py
#        \author   chenghuige  
#          \date   2022-05-15 07:00:15.332073
#   \Description  
# ==============================================================================

  
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os, sys
if os.path.exists('/kaggle'):
  sys.path.append('/kaggle/input/pikachu/utils')
  sys.path.append('/kaggle/input/pikachu/third')
  sys.path.append('.')
else:
  sys.path.append('..')
  sys.path.append('../../../../utils')
  sys.path.append('../../../../third')

from gezi.common import *
import gc
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["NCCL_DEBUG"] = 'WARNING'

from src.config import *
from src.preprocess import *
from src.postprocess import *

flags.DEFINE_string('ofile', '../working/x.pkl', '')
flags.DEFINE_bool('infer_norm', False, '')

# ic = print
def main(argv):
  ic('infer.py start')
  model_dir = FLAGS.model_dir
  bs = FLAGS.eval_bs
  out_file = FLAGS.ofile
  infer_norm = FLAGS.infer_norm
  ic(bs)
  gezi.restore_configs(model_dir, ignores=gezi.get_commandline_flags())
  FLAGS.mns = ''
  FLAGS.train_allnew = False
  FLAGS.grad_acc = 1
  FLAGS.restore_configs = False
  FLAGS.bs, FLAGS.eval_bs = bs, bs
  FLAGS.num_gpus = 1
  mt.init()
  FLAGS.model_dir = model_dir
  FLAGS.pymp = False
  FLAGS.num_workers = 2
  FLAGS.pin_memory = True
  FLAGS.persistent_workers = True
  FLAGS.workers = 1
  FLAGS.backbone_dir = None
  FLAGS.hack_rate = 1.
  FLAGS.resolve_encodings = True
  FLAGS.mlabel_cv = False

  if gezi.in_kaggle():
    backbone = FLAGS.backbone.split('/')[-1].replace('_', '-')
    FLAGS.backbone = '../input/' + backbone
  
  show()
  FLAGS.mode = 'test'
  FLAGS.work_mode = 'test'
  FLAGS.model_dir = model_dir
  ic(FLAGS.backbone, FLAGS.model_dir, os.path.exists(f'{FLAGS.model_dir}/model.pt'))
  try:
    display(pd.read_csv(f'{model_dir}/metrics.csv'))
  except Exception as e:
    logger.warning(e)

  logger.info('before get test_ds')
  test_ds = get_dataloaders(test_only=True)
  logger.info('after get test_ds')

  from src.torch.model import Model
  model = Model()
  
  logger.info('before load weights')
  ic(gezi.get_mem_gb())
  gezi.load_weights(model, model_dir)
  ic(gezi.get_mem_gb())
  logger.info('load weights done')

  logger.info('before predict')
  x = lele.predict(model, test_ds, out_keys=['text_id', 'label'], amp=True, fp16=True)
  logger.info('predict done')

  x['pred'] = to_pred(x['pred'])
  if 'label' in x:
    x['label'] = to_pred(x['label'])
  ic(len(x['pred']))
  gezi.save(x, out_file)
  
  gc.collect()
  torch.cuda.empty_cache()

if __name__ == '__main__':
  app.run(main)  

In [ ]:
models = {}
def show_models(indexes, show=False):
  for i in indexes:
    model_dir = f'../input/{MODEL_NAME}-model{i}'
    model_path = open(f'{model_dir}/path.txt').readline().strip()
    models[i] = model_path
  ic({i: models[i] for i in indexes})
  assert len(models) == len(set(models.values()))

def get_bs(model_name):
  bs = 64
  if 'len768' in model_name:
    bs = 16
  if 'len1024' in model_name:
    bs = 16
  if 'len1280' in model_name:
    bs = 8
  if 'listwise' in model_name:
    bs = 2
  bs *= 4
  return bs
        
def work(ensembler, indexes, weight=1.):
  for i in tqdm(indexes):
    model_dir = f'../input/{MODEL_NAME}-model{i}'
    bs = get_bs(models[i])
    ofile = '../working/x.pkl'
    command = f'python ./src/infer.py --model_dir={model_dir} --eval_bs={bs} --ofile={ofile} --hack_infer={TEST_MODE}'
    ic(i, model_dir, bs, command, weight)
    os.system(command)
    os.system('rm -rf ../working/online/*')
    x = gezi.load(ofile)
    ic(gezi.get_mem_gb())
    ensembler.add(x, weight)

In [ ]:
IDX = 9
ws = [[3.76528658e-01, 8.76028883e-02, 4.10500987e-02, 3.22930013e-01,
        2.03192569e-02, 1.13337706e-01],
       [3.49303631e-02, 9.99559246e-01, 3.57892307e-01, 1.20501608e-04,
        3.34969044e-01, 6.38790611e-01],
       [3.90324865e-04, 1.11180995e-01, 1.00381314e-03, 1.72253285e-01,
        1.84132911e-01, 8.79218765e-04],
       [2.44003000e-04, 2.85680342e-03, 6.53105950e-02, 2.21457610e-02,
        1.16445854e-01, 3.36888397e-01],
       [8.09464853e-01, 1.72158732e-02, 2.52355005e-01, 3.31848717e-02,
        1.92769551e-02, 9.33186377e-01],
       [9.78288281e-02, 1.45423258e-03, 5.73361427e-02, 5.63644383e-02,
        2.45430371e-01, 1.75666298e-02],
       [5.92361615e-02, 1.17123167e-01, 5.47863184e-01, 9.99272338e-01,
        9.42910343e-01, 3.67106373e-01],
       [9.83205055e-01, 3.76524985e-01, 9.32257497e-01, 4.75461323e-02,
        7.78012503e-05, 2.36276564e-01],
       [2.05574885e-02, 1.66367010e-01, 5.48905473e-02, 3.97917136e-01,
        4.15895033e-02, 5.59219706e-01],
       [1.08708590e-01, 1.43155107e-01, 1.63345411e-01, 1.14435784e-01,
        2.07771120e-02, 5.95015375e-02]]

In [ ]:
gezi.init_flags()
for i, indexes in enumerate(INDEXES_LIST):
  ic(i, indexes, WEIGHTS[i])
  show_models(indexes)

# ensembler = gezi.Ensembler(need_sort=True, id_key='text_id', includes=['pred', 'label'])
# for i, indexes in tqdm(enumerate(INDEXES_LIST), total=len(INDEXES_LIST)):
#   ic(i, indexes, WEIGHTS[i])
#   show_models(indexes)
#   work(ensembler, indexes, WEIGHTS[i])
# x = ensembler.finalize()

# ensembler2 = gezi.Ensembler(includes=['pred', 'label'], inplace=False)
# for fold in range(FOLDS):
#     ensembler = gezi.Ensembler(includes=['pred', 'label'], inplace=False)
#     for i in range(NUM_MODELS):
#       idx = fold * NUM_MODELS + i
#       if i < 1:
#         work(ensembler, [idx], np.array([1,1,1,1,1,1], float))
#       elif i < 2: 
#         work(ensembler, [idx], np.array([1,1,10,1,1,1], float) * 2)
#       elif i < 3: 
#         work(ensembler, [idx], np.array([1,1,1,1,1,1], float) * 0.25)
#       elif i < 4: 
#         work(ensembler, [idx], np.array([1,1,1,1,1,1], float))
#       elif i < 5: 
#         work(ensembler, [idx], np.array([2,1,10,1,1,2], float) * 2)
#       elif i < 6: 
#         work(ensembler, [idx], np.array([1,1,1,1,10,1], float) * 0.25)
#       else:
#         work(ensembler, [idx], np.array([1,1,10,1,0.25,0.25], float) * 0.8)
#     x = ensembler.finalize()
#     if fold == 0:
#       ensembler2.add(x, 2)
#     else:
#       ensembler2.add(x, 1)
# x = ensembler2.finalize()

ensembler2 = gezi.Ensembler(includes=['pred', 'label'], inplace=False)
for fold in range(FOLDS):
    ensembler = gezi.Ensembler(includes=['pred', 'label'], inplace=False)
    for i in range(NUM_MODELS):
      idx = fold * NUM_MODELS + i
      if i < IDX:
        work(ensembler, [idx], np.array(ws[i]))
      else:
        work(ensembler, [idx], np.array(ws[IDX]))
    x = ensembler.finalize()
    if fold == 0:
      ensembler2.add(x, 2)
    else:
      ensembler2.add(x, 1)
x = ensembler2.finalize()

In [ ]:
if TEST_MODE:
  from src.eval import calc_metric
  ic(calc_metric(np.vstack(x['label']), np.vstack(x['pred'])))

In [ ]:
x['pred'] = np.clip(x['pred'], 1.0, 5.0)

In [ ]:
from src.eval import to_df
df = to_df(x)

In [ ]:
os.system('rm -rf ./*')

In [ ]:
df.to_csv('./submission.csv', index=False)

In [ ]:
pd.read_csv('./submission.csv')